`Notebook Description:` contains different methods for generating forcasting datasets

## Setup 

In [ ]:
import os 
import pandas as pd
import numpy as np 
import random
import matplotlib.pyplot as plt

In [17]:
# Defining directory paths
forecast_data_dir = '../data/generated/forecast/'
inventory_data_dir = '../data/generated/inventory/'
results_data_dir = '../data/generated/Results_DFC/'

## A days forward coverage function 
that takes both datasets and returns the days forward coverage for each product.

- The Days Forward Coverage (DFC) function calculates, for each product, the number of days into the future that the current inventory can satisfy the forecasted demand, considering the EXPIRY_DATEs of the inventory batches.

### Function Steps

- **Step 1: Data Preparation**
- **Step 2: Iterate Over Each Product**
- **Step 3: Simulate Inventory Consumption**
  
    For each date in the forecasted sales data for the product:
    1. **Check for Expired Inventory**:
      - Remove batches from `remaining_inventory` that have expired before the current date.
    2. **Determine Demand**:
      - Get the forecasted sales (demand) for the current date.
    3. **Consume Inventory**:
      - While `demand > 0` and `remaining_inventory` is not empty:
        - Use the batch with the earliest EXPIRY_DATE.
        - Calculate the quantity to consume from the batch (up to the batch's available inventory).
        - Reduce `demand` and the batch's `INVENTORY` accordingly.
        - Remove the batch if its `INVENTORY` drops to zero.
    4. **Update DFC**:
   
         - If `demand == 0`:
           - Increment `DFC` by 1 (full day covered).
         - If `demand > 0`:
           - Calculate the fraction of the day covered:
             ```
             fraction = inventory_consumed / forecasted_demand
             ```
           - Increment `DFC` by the fraction.
           - Break the loop as inventory is depleted.

- **Step 4: Return the Results**


In [15]:
def calculate_days_forward_coverage(forecast_df, inventory_df):
    """
    Calculate Days Forward Coverage (DFC) for each product.

    Parameters:
    - forecast_df (DataFrame): Forecasted sales data with columns ['PRODUCT_ID', 'DATE', 'FORECASTED SALES']
    - inventory_df (DataFrame): Inventory data with columns ['PRODUCT_ID', 'BATCH_ID', 'EXPIRY_DATE', 'INVENTORY']

    Returns:
    - dfc_df (DataFrame): DataFrame with columns ['PRODUCT_ID', 'DAYS FORWARD COVERAGE']
    """
    date_format = '%d/%m/%Y'

    # Parse dates in forecast_df
    forecast_df['DATE'] = pd.to_datetime(forecast_df['DATE'], format=date_format, errors='coerce')

    # Parse dates in inventory_df
    inventory_df['EXPIRY_DATE'] = pd.to_datetime(inventory_df['EXPIRY_DATE'], format=date_format, errors='coerce')

    # # Step 1: Data Preparation
    # forecast_df['DATE'] = pd.to_datetime(forecast_df['DATE'], dayfirst=True)
    # inventory_df['EXPIRY_DATE'] = pd.to_datetime(inventory_df['EXPIRY_DATE'], dayfirst=True)
    
    # Sort data
    forecast_df.sort_values(['PRODUCT_ID', 'DATE'], inplace=True)
    inventory_df.sort_values(['PRODUCT_ID', 'EXPIRY_DATE'], inplace=True)
    
    
    results = []

    # Get list of unique PRODUCT_IDs
    product_ids = inventory_df['PRODUCT_ID'].unique()
    
    # Step 3: Iterate Over Each Product
    for product_id in product_ids:
        # Extract product data
        product_inventory = inventory_df[inventory_df['PRODUCT_ID'] == product_id].copy()
        product_forecast = forecast_df[forecast_df['PRODUCT_ID'] == product_id].copy()
        
        # If there's no forecast data for the product, skip
        if product_forecast.empty:
            results.append({'PRODUCT_ID': product_id, 'DAYS FORWARD COVERAGE': 0})
            continue
        
        # Initialize variables
        # Convert inventory to a list of dicts for easy manipulation
        inventory_batches = product_inventory.to_dict('records')
        dfc = 0  # Days Forward Coverage
        inventory_depleted = False
        
        # Prepare inventory batches
        # Start with earliest EXPIRY_DATE
        inventory_batches.sort(key=lambda x: x['EXPIRY_DATE'])
        
        # Initialize index for inventory batches
        batch_index = 0
        
        # Step 4: Simulate Inventory Consumption
        for forecast_date, demand in zip(product_forecast['DATE'], product_forecast['FORECASTED_SALES']):
            # Remove expired inventory batches
            while batch_index < len(inventory_batches) and inventory_batches[batch_index]['EXPIRY_DATE'] < forecast_date:
                batch_index += 1
            
            # Check if inventory is depleted
            if batch_index >= len(inventory_batches):
                inventory_depleted = True
                break  # No more inventory available
            
            remaining_demand = demand
            inventory_consumed = 0
            
            # Consume inventory batches
            current_batch_idx = batch_index
            while remaining_demand > 0 and current_batch_idx < len(inventory_batches):
                batch = inventory_batches[current_batch_idx]
                available_qty = batch['INVENTORY']
                
                # Calculate quantity to consume
                consume_qty = min(available_qty, remaining_demand)
                
                # Update batch inventory
                batch['INVENTORY'] -= consume_qty
                remaining_demand -= consume_qty
                inventory_consumed += consume_qty
                
                # Move to next batch if current batch is depleted
                if batch['INVENTORY'] == 0:
                    current_batch_idx += 1
                else:
                    break  # Current batch still has inventory
                
            # Update batch index
            batch_index = current_batch_idx
            
            if remaining_demand == 0:
                # Full day covered
                dfc += 1
            else:
                # Partial day covered
                fraction_covered = inventory_consumed / demand
                dfc += fraction_covered
                inventory_depleted = True
                break  # Inventory depleted
            
        # If inventory remains but no more forecasted dates
        if not inventory_depleted and dfc < len(product_forecast):
            # Continue adding days until inventory depletes or forecast ends
            # Here, since forecast data has ended, we cannot project further
            pass  # DFC remains as calculated
        
        # Store result
        results.append({'PRODUCT_ID': product_id, 'DAYS FORWARD COVERAGE': round(dfc, 2)})
    
    # Step 6: Return the Results
    dfc_df = pd.DataFrame(results)
    
    return dfc_df

**Constraints:**

- The function does not rely on any specifics of the datasets beyond the column names and expected data types.
- It can handle any number of products, dates, and batches.
- The function accounts for partial day coverage when inventory cannot meet the full day's demand.
- Once the inventory is depleted, the function stops calculating DFC for that product.
- If there is no forecast data for a product, the DFC is set to 0.
- If a product has forecast data but no inventory, it will be processed, and the DFC will be 0 since there's no inventory to meet the demand.


## Applying it over generated datasets

In [6]:
forecast_csv_files = [os.path.join(forecast_data_dir, file) for file in os.listdir(forecast_data_dir) if file.endswith('.csv')]
inventory_csv_files = [os.path.join(inventory_data_dir, file) for file in os.listdir(inventory_data_dir) if file.endswith('.csv')]
inventory_csv_files

['../data/generated/inventory/method_1_uniform_random_inventory.csv',
 '../data/generated/inventory/method_2_random_ND_inventory.csv',
 '../data/generated/inventory/method_3_seasonality_inventory.csv']

In [13]:
forecast_dfs = [pd.read_csv(file) for file in forecast_csv_files]
inventory_dfs = [pd.read_csv(file) for file in inventory_csv_files]

In [19]:
counter = 1 
for forecast_df, inventory_df in zip(forecast_dfs, inventory_dfs): 
    results_file_path = os.path.join(results_data_dir, f'dfc_results_method{counter}.csv')
    dfc_results = calculate_days_forward_coverage(forecast_df, inventory_df)
    dfc_results.to_csv(results_file_path, index=False)
    counter += 1

#### Review the results

In [22]:
df_method1 = pd.read_csv(results_data_dir+'dfc_results_method1.csv') 
df_method1.head()

,PRODUCT_ID,DAYS FORWARD COVERAGE
0,1,1.95
1,2,3.67
2,3,1.67
3,4,2.53
4,5,4.04


In [34]:
df_method1.describe()

,PRODUCT_ID,DAYS FORWARD COVERAGE
count,1000.000000,1000.000000
mean,500.500000,3.377910
std,288.819436,1.289534
min,1.000000,0.700000
25%,250.750000,2.427500
50%,500.500000,3.190000
75%,750.250000,4.132500
max,1000.000000,9.030000


In [23]:
df_method2 = pd.read_csv(results_data_dir+'dfc_results_method2.csv') 
df_method2.head()

,PRODUCT_ID,DAYS FORWARD COVERAGE
0,1,6.47
1,2,5.88
2,3,9.17
3,4,8.23
4,5,6.92


In [35]:
df_method2.describe()

,PRODUCT_ID,DAYS FORWARD COVERAGE
count,1000.000000,1000.000000
mean,500.500000,12.617390
std,288.819436,6.943239
min,1.000000,3.670000
25%,250.750000,7.485000
50%,500.500000,10.260000
75%,750.250000,15.530000
max,1000.000000,31.000000


In [24]:
df_method3 = pd.read_csv(results_data_dir+'dfc_results_method3.csv') 
df_method3.head()

,PRODUCT_ID,DAYS FORWARD COVERAGE
0,1,28.23
1,2,9.50
2,3,4.82
3,4,6.82
4,5,7.62


In [36]:
df_method3.describe()

,PRODUCT_ID,DAYS FORWARD COVERAGE
count,1000.000000,1000.000000
mean,500.500000,9.135910
std,288.819436,8.149778
min,1.000000,1.370000
25%,250.750000,3.580000
50%,500.500000,5.835000
75%,750.250000,11.202500
max,1000.000000,31.000000
